### Data Cleaning & Preprocessing

Now that everything’s merged — demographics, biometrics, medical history — it’s time to clean. Not the most exciting part, but one of the most important.

Right away, I can see a few issues: some missing values we’ll need to handle carefully, a diagnosis_date column that’s likely still a string, and possibly a few strange entries in cost. Oh, and we should double-check for duplicate patient IDs just to be safe.

Let’s take it from the top and clean as we go.

In [13]:
## import necessary libraries
import pandas as pd

## Load merged dataset 
merged_df = pd.read_csv("Merged_Healthcare_Data.csv")

# Just checking the shape and structure
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33146 entries, 0 to 33145
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   patient_id                33146 non-null  object 
 1   diagnosis                 30000 non-null  object 
 2   diagnosis_date            30000 non-null  object 
 3   visit_type                30000 non-null  object 
 4   cost                      29850 non-null  float64
 5   bmi                       23913 non-null  float64
 6   blood_pressure_systolic   24155 non-null  float64
 7   blood_pressure_diastolic  24155 non-null  float64
 8   cholesterol_total         23968 non-null  float64
 9   age                       32821 non-null  float64
 10  gender                    32982 non-null  object 
 11  ethnicity                 33146 non-null  object 
 12  zip_code                  33146 non-null  int64  
dtypes: float64(6), int64(1), object(6)
memory usage: 3.3+ MB


In [14]:
# Step 2: Convert 'diagnosis_date' to proper datetime
# We'll need this to analyze disease trends over time later

merged_df['diagnosis_date'] = pd.to_datetime(merged_df['diagnosis_date'], errors='coerce') ## Converts the values in the 'diagnosis_date' column from strings into proper datetime objects using pandas.to_datetime(); errors= 'coerce' tp replace the blank or misspelled value with NAT

In [15]:
# Step 3: Before we clean or drop anything, let’s take stock of what's missing.
missing = merged_df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
print("Missing values summary:\n", missing)

Missing values summary:
 bmi                         9233
cholesterol_total           9178
blood_pressure_systolic     8991
blood_pressure_diastolic    8991
cost                        3296
diagnosis                   3146
diagnosis_date              3146
visit_type                  3146
age                          325
gender                       164
dtype: int64


### What We're Seeing in the Missing Values

Most of the missing data is in biometrics — patients who haven’t had screenings.
A second major block is medical records — patients who show biometric risk but haven’t been diagnosed yet. These are *exactly* who Loblaw might want to flag.
A few gaps in age/gender — we’ll flag these, not drop them.
So in short, we’re keeping everything, but tagging it carefully so we can filter it smartly later.

In [16]:
#step 4 We're not dropping these patients — they might still show signs of risk or cost
merged_df['age_missing'] = merged_df['age'].isnull()
merged_df['gender_missing'] = merged_df['gender'].isnull()
merged_df['ethnicity_missing'] = merged_df['ethnicity'].isnull()

In [18]:
###Step 5: Normalize Gender (make sure 'male', 'MALE', ' Male ' all match)
merged_df['gender'] = merged_df['gender'].str.strip().str.capitalize() ## .str.strip()-> remove any leading spaces from each value;.str.capitalize()-> converts the first letter to uppercase

In [19]:
# Step 6:  Keep nulls they might be legitimate unbilled visits.
# Remove cost = 0 or < 0, likely data entry or system error
merged_df = merged_df[(merged_df['cost'].isnull()) | (merged_df['cost'] > 0)]

In [20]:
##Step 7: Check for and Drop Exact Duplicate Rows
duplicates = merged_df.duplicated()
print("Duplicate rows found:", duplicates.sum())

Duplicate rows found: 0


In [28]:
##step 8:Save Cleaned Dataset
merged_df.to_csv("Cleaned_Merged_Data.csv", na_rep='NaN', index=False)

## Summary of the Data cleaning and preprocessing 
The goal here wasn’t to make everything perfect — just usable and meaningful.

Not every missing value needed fixing. Some of them told a story: patients who never showed up for screening, or who’ve had biometric flags but never been officially diagnosed. Those might be the people we most want to pay attention to.

So I didn’t over-clean. I flagged what’s missing, fixed what would break things later (like cost errors or duplicate rows), and left the rest alone, for now.

This gives us a dataset that’s honest. Not flawless, but functional. And ready for the next step: actually learning something from it.